# PyFIM - Coop Case Study - Complementary and Substitute Products 

## Complementary and Substitute Products Products PyFIM

## Preprocessing

In [1]:
# Libraries 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pylab as pylab
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.metrics import mean_squared_error as mse 
from sklearn.neural_network import MLPRegressor
from apyori import apriori ## Importing apriori package
from fim import arules
#from mlxtend.frequent_patterns import apriori, association_rules

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (22, 14),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}

pylab.rcParams.update(params)

#### Import functions from ./Functions_Forecasting AR file in this same folder, look for comments on the functions in that notebook



In [2]:
from ipynb.fs.defs.Functions_Forecasting import * 


### importing data

In [3]:
data = pd.read_csv("./4months_v2/rl_4months.csv")

In [4]:
data.shape

(3221229, 40)

In [5]:
#data[["receiptKey","DayDate","hourOfDay","minuteOfHour","quantity","ItemID"]].iloc[0:20,]

In [6]:
# Generate unique name 
data['Unique_Item_Name'] = data['ItemCategoryName'] + '_' + data['ItemSegmentName']

In [24]:
data_subset = data[['customerId', 'DayDate','hourOfDay', 'ItemID', 'quantity','Unique_Item_Name', 'CoopOnlineYN']]


In [26]:
data_subset.head()

,customerId,DayDate,hourOfDay,ItemID,quantity,Unique_Item_Name,CoopOnlineYN
0,155843,2020-02-05,11,277403,1.0,GRÖNSAKER_SKUREN SALLAD,N
1,155843,2020-02-05,11,322,3.0,BUTIKSBAKATBRÖD&_WIENERBRÖD&CROISSANT,N
2,155843,2020-02-05,11,322,0.0,BUTIKSBAKATBRÖD&_WIENERBRÖD&CROISSANT,N
3,155843,2020-02-05,11,303321,1.0,MEJERI_FILMJÖLK,N
4,155843,2020-02-05,11,601635,1.0,FÄRDIGLAGAD MAT_TILLBEHÖR,N


### Deletion of returns and false duplicates of Negative and 0 Quantities

Account for only online sales

In [30]:
data_subset = data_subset[data_subset['CoopOnlineYN'] == 'Y']

In [31]:
data_subset = data_subset[data_subset.quantity>0]

In [32]:
data_subset['timestamp'] = data_subset['DayDate'].astype(str) + ' '+  data_subset['hourOfDay'].astype(str)


### Aggregating Datasets

In [34]:
data_subset_sub = data_subset[['customerId', 'ItemID']]
data_subset_sub

,customerId,ItemID
4292,693118,276046
4294,693118,303939
4295,693118,303407
4296,693118,306629
4298,693118,61323
...,...,...
3221122,4257298,507615
3221123,4257298,277284
3221124,4257298,292849
3221125,4257298,303929


In [51]:
transactions = [a[1]['ItemID'].tolist() for a in list(data_subset_sub.groupby(['customerId']))] ## Combing all the items in list format for each cutomer

In [37]:
hash_map_series = data_subset.groupby(['ItemID']).first()['Unique_Item_Name']

In [38]:
hash_map_series = pd.DataFrame(hash_map_series.str.strip())# Remove 

In [39]:
hash_map_series

,Unique_Item_Name
ItemID,
2922,FIKABRÖD_0
10914,KROPPSVÅRD_KVINNA
11501,FÄRDIGMAT & FISK_KÖTTKONSERVER
11503,GODIS_STYCKSAKER
11541,GODIS_STYCKSAKER
...,...
751980,GRÖNSAKER_0
779855,GRÖNSAKER_0
781191,FRUKT & BÄR_0


In [40]:
??arules

## PyFIM

In [102]:
# inputs
supp = 2 # minimum support of an assoc. rule (default: 10)
conf = 60 # minimum confidence of an assoc. rule (default: 80%)
report = 'asC'

In [103]:
# make dict for nicer looking column names
report_colnames = {
    'a': 'support_itemset_absolute',
    's': 'support_itemset_relative',
    'S': 'support_itemset_relative_pct',
    'b': 'support_bodyset_absolute',
    'x': 'support_bodyset_relative',
    'X': 'support_bodyset_relative_pct',
    'h': 'support_headitem_absolute',
    'y': 'support_headitem_relative',
    'Y': 'support_headitem_relative_pct',
    'c': 'confidence',
    'C': 'confidence_pct',
    'l': 'lift',
    'L': 'lift_pct',
    'e': 'evaluation',
    'E': 'evaluation_pct',
    'Q': 'xx',
    'S': 'support_emptyset',
    }

In [104]:
result = arules(transactions, supp=supp, conf=conf, report=report)

In [105]:
colnames = ['consequent', 'antecedent'] + [report_colnames.get(k, k) for k in list(report)]
df_rules = pd.DataFrame(result, columns=colnames)
df_rules = df_rules.sort_values('support_itemset_absolute', ascending=False)
print(df_rules.shape)


(35113, 5)


In [106]:
results_df_names.head()

,consequent,antecedent,support_itemset_absolute,support_itemset_relative,confidence_pct,consequent names,antecedent_names
0,46877,(),5002,0.908299,90.829853,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,
1,46877,"(279105,)",2204,0.400218,92.449664,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[IN&UTBETAL MOMSF_MOMSFRI -ÖVR]
2,46877,"(64582,)",2143,0.389141,91.895369,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[FRUKT & BÄR_KOKBANANER]
3,46877,"(62104,)",1433,0.260214,92.095116,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[GRÖNSAKER_GUL LÖK]
4,46877,"(279106,)",1353,0.245687,92.989691,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[IN&UTBETAL MOMSF_MOMSFRI -ÖVR]


In [79]:
results_df_names = df_rules.merge(hash_map_series, left_on='consequent', right_on='ItemID').rename(columns= {'Unique_Item_Name': 'consequent names'})


In [80]:
results_df_names = results_df_names.dropna()

In [81]:
results_df_names['antecedent_names'] = results_df_names.antecedent.apply(lambda x:"" if x== () else [hash_map_series.loc[i] for i in x])

In [82]:
results_df_names 

,consequent,antecedent,support_itemset_absolute,support_itemset_relative,confidence_pct,consequent names,antecedent_names
0,46877,(),5002,0.908299,90.829853,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,
1,46877,"(279105,)",2204,0.400218,92.449664,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[[IN&UTBETAL MOMSF_MOMSFRI -ÖVR]]
2,46877,"(64582,)",2143,0.389141,91.895369,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[[FRUKT & BÄR_KOKBANANER]]
3,46877,"(62104,)",1433,0.260214,92.095116,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[[GRÖNSAKER_GUL LÖK]]
4,46877,"(279106,)",1353,0.245687,92.989691,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[[IN&UTBETAL MOMSF_MOMSFRI -ÖVR]]
...,...,...,...,...,...,...,...
35108,62062,"(278598, 64660, 273860, 64582)",73,0.013256,60.833333,GRÖNSAKER_GURKA,"[[GRÖNSAKER_PAPRIKA RÖD], [GRÖNSAKER_GURKA], [..."
35109,62062,"(61807, 62132, 62104, 279105)",69,0.012530,60.526316,GRÖNSAKER_GURKA,"[[GRÖNSAKER_ISBERGSSALLAD], [GRÖNSAKER_LÖSVIKT..."
35110,62062,"(61807, 61818, 64582, 279105)",68,0.012348,60.714286,GRÖNSAKER_GURKA,"[[GRÖNSAKER_ISBERGSSALLAD], [GRÖNSAKER_RÖD LÖK..."
35111,278556,"(62047, 62079, 61960, 46877)",69,0.012530,61.061947,GRÖNSAKER_BROCCOLI FÄRSK,"[[GRÖNSAKER_BLOMKÅL], [GRÖNSAKER_LÖK BLAD], [F..."


# Eclat Algorithm PyFIM Libary

In [84]:
from fim import eclat

In [87]:
# inputs
supp = 2 # minimum support of an assoc. rule (default: 10)
conf = 60 # minimum confidence of an assoc. rule (default: 80%)
report = 'asC'

In [88]:
result = eclat(transactions, supp=supp, conf=conf, report=report)

In [89]:
# make dict for nicer looking column names
report_colnames = {
    'a': 'support_itemset_absolute',
    's': 'support_itemset_relative',
    'S': 'support_itemset_relative_pct',
    'b': 'support_bodyset_absolute',
    'x': 'support_bodyset_relative',
    'X': 'support_bodyset_relative_pct',
    'h': 'support_headitem_absolute',
    'y': 'support_headitem_relative',
    'Y': 'support_headitem_relative_pct',
    'c': 'confidence',
    'C': 'confidence_pct',
    'l': 'lift',
    'L': 'lift_pct',
    'e': 'evaluation',
    'E': 'evaluation_pct',
    'Q': 'xx',
    'S': 'support_emptyset',
    }

In [90]:
result = arules(transactions, supp=supp, conf=conf, report=report)

In [91]:
colnames = ['consequent', 'antecedent'] + [report_colnames.get(k, k) for k in list(report)]
df_rules = pd.DataFrame(result, columns=colnames)
df_rules = df_rules.sort_values('support_itemset_absolute', ascending=False)
print(df_rules.shape)


(35113, 5)


In [92]:
results_df_names.head()

,consequent,antecedent,support_itemset_absolute,support_itemset_relative,confidence_pct,consequent names,antecedent_names
0,46877,(),5002,0.908299,90.829853,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,
1,46877,"(279105,)",2204,0.400218,92.449664,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[[IN&UTBETAL MOMSF_MOMSFRI -ÖVR]]
2,46877,"(64582,)",2143,0.389141,91.895369,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[[FRUKT & BÄR_KOKBANANER]]
3,46877,"(62104,)",1433,0.260214,92.095116,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[[GRÖNSAKER_GUL LÖK]]
4,46877,"(279106,)",1353,0.245687,92.989691,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[[IN&UTBETAL MOMSF_MOMSFRI -ÖVR]]


In [93]:
results_df_names = df_rules.merge(hash_map_series, left_on='consequent', right_on='ItemID').rename(columns= {'Unique_Item_Name': 'consequent names'})


In [94]:
results_df_names = results_df_names.dropna()

In [97]:
results_df_names['antecedent_names'] = results_df_names.antecedent.apply(lambda x:"" if x== () else [hash_map_series.loc[i][0] for i in x])

In [98]:
results_df_names 

,consequent,antecedent,support_itemset_absolute,support_itemset_relative,confidence_pct,consequent names,antecedent_names
0,46877,(),5002,0.908299,90.829853,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,
1,46877,"(279105,)",2204,0.400218,92.449664,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[IN&UTBETAL MOMSF_MOMSFRI -ÖVR]
2,46877,"(64582,)",2143,0.389141,91.895369,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[FRUKT & BÄR_KOKBANANER]
3,46877,"(62104,)",1433,0.260214,92.095116,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[GRÖNSAKER_GUL LÖK]
4,46877,"(279106,)",1353,0.245687,92.989691,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[IN&UTBETAL MOMSF_MOMSFRI -ÖVR]
...,...,...,...,...,...,...,...
35108,62062,"(278598, 64660, 273860, 64582)",73,0.013256,60.833333,GRÖNSAKER_GURKA,"[GRÖNSAKER_PAPRIKA RÖD, GRÖNSAKER_GURKA, GRÖNS..."
35109,62062,"(61807, 62132, 62104, 279105)",69,0.012530,60.526316,GRÖNSAKER_GURKA,"[GRÖNSAKER_ISBERGSSALLAD, GRÖNSAKER_LÖSVIKT, G..."
35110,62062,"(61807, 61818, 64582, 279105)",68,0.012348,60.714286,GRÖNSAKER_GURKA,"[GRÖNSAKER_ISBERGSSALLAD, GRÖNSAKER_RÖD LÖK, F..."
35111,278556,"(62047, 62079, 61960, 46877)",69,0.012530,61.061947,GRÖNSAKER_BROCCOLI FÄRSK,"[GRÖNSAKER_BLOMKÅL, GRÖNSAKER_LÖK BLAD, FRUKT ..."
